In [ ]:
!pip install torch transformers spacy openai-whisper speechrecognition gtts googletrans==4.0.0-rc1 langdetect
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 86.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, pipeline
import spacy
import speech_recognition as sr
import pyttsx3
from langdetect import detect
from googletrans import Translator

# Load models
model_name = "facebook/blenderbot-400M-distill"  # Better chatbot model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

sentiment_analyzer = pipeline("text-classification", model="distilbert-base-uncased-finetuned-sst-2-english")
nlp = spacy.load("en_core_web_sm")
translator = Translator()

tts_engine = pyttsx3.init()
tts_engine.setProperty("rate", 150)

def detect_sentiment(text):
    return sentiment_analyzer(text)[0]['label']

def extract_subject(text):
    doc = nlp(text)
    for token in doc:
        if token.dep_ in ["nsubj", "pobj"]:
            return token.text
    return None

def generate_response(user_input):
    sentiment = detect_sentiment(user_input)
    subject = extract_subject(user_input)

    if subject and "I" in user_input:
        user_input = user_input.replace("I", f"My {subject}")

    inputs = tokenizer(user_input, return_tensors="pt", truncation=True)
    output_ids = model.generate(**inputs, max_length=50)
    response = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    if sentiment == "NEGATIVE" and not response.startswith("I'm sorry"):
        response = "I'm sorry to hear that. " + response
    elif sentiment == "POSITIVE" and not response.startswith("That sounds great"):
        response = "That sounds great! " + response

    return response

def chatbot():
    print("Chatbot is ready! Type 'exit' to stop.")
    while True:
        user_input = input("You: ")
        if user_input.lower() == "exit":
            break
        response = generate_response(user_input)
        print("Chatbot:", response)
        tts_engine.say(response)
        tts_engine.runAndWait()

chatbot()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Device set to use cuda:0


RuntimeError: This means you probably do not have eSpeak or eSpeak-ng installed!